In [ ]:
import numpy as np
from scipy.optimize import fminbound
from scipy import interp

def bellman_operator(w, grid, beta, u, f, shocks, Tw=None, compute_policy=0):
    """
    the approximate bellman operator, which computes and returns the
    updated value function Tw on the grid points. an array to store
    the new set of values Tw is optionally supplied (to avoid having to
    allocate new arrays at each iteration). if supplied, any existing data in
    Tw will be overwritten
    
    parameters
    ---------
    w: array_like(float, ndim=1)
        the value of the input function on different grid points
    grid: array_like(float, ndim=1)     
        the set of grid points
    u: function
        the utility function
    f: function
        the production function
    shocks: numpy array
        an array of draws from the shock, fro monte carlo integration(to
        compute expectations)
    beta: scalar
        the discount factor
    Tw: array_like(float, ndim=1) optional(default=None)
        array to write output values to
    compute_policy: Boolean, optional(default=False)
        whether or not to compute policy function
    """
    
    #===apply linear interpolation to w===#
    w_func=lambda x: interp(x, grid, w)
    
    #===initialize Tw if necessary==#
    if Tw is None:
        Tw=np.empty(len(w))
        
    if compute_policy:
        sigma=np.empty(len(w))
    
    #==set Tw[i]=max_c{u(c)+beta E w(f(y-c) z)}==#
    for i, y in enumerate(grid):
        def objective(c):
            return -u(c)-beta*np.mean(w_func(f(y-c)*shocks))
        c_star=fminbound(objective, 1e-10, y)
        if compute_policy:
            sigma[i]=c_star
        Tw[i]=-objective(c_star)
        
    if compute_policy:
        return Tw, sigma
    else:
        return Tw